In [1]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import os
import pandas as pd

from datetime import datetime
from datetime import timedelta


In [2]:

Year=2019
Mounth=2
Day=14
Hour=13
j=0
T=60


T_ON = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time i
T_OFF = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time 
X_aux = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time i

# Especifique o diretório onde estão seus arquivos Excel.
diretorio = 'C:/Users/anaol/Documents/NexIK_Flexibility/Data_Valores Reais'

# Lista todos os arquivos na pasta.
arquivos_na_pasta = os.listdir(diretorio)

In [3]:
def function_Ton_T_off (df, j, nome_do_arquivo):

    if nome_do_arquivo in arquivos_na_pasta:
        caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
        
        # Use a função read_excel para carregar os dados em um DataFrame.
        df = pd.read_csv(caminho_arquivo, sep=';')

        # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
    else:
        print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

    df['first_min_on'] = df['first_min_on'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S+00:00"))
    df['last_min_on'] = df['last_min_on'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S+00:00"))
    df['timedelta_on'] = df['timedelta_on'].apply(lambda x: datetime.strptime(x.replace(' ', '').split('days')[1], '%H:%M:%S'))


    for k in range(len(df)):
        if df['first_min_on'][k].second == 30:
            df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
        if df['last_min_on'][k].second == 30:
            df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
        if df['timedelta_on'][k].second == 30:
            df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
        


    for i in range (T):
        for k in range(len(df)):
            if df['first_min_on'][k].year==Year and df['first_min_on'][k].month ==Mounth and df['first_min_on'][k].day == Day and df['first_min_on'][k].hour == Hour:

                if i < df['first_min_on'][k].minute:
                    X_aux[i][j] = 0
                    break
                elif i >=  df['first_min_on'][k].minute and i <=  df['last_min_on'][k].minute and df['last_min_on'][k].hour == Hour:
                    X_aux[i][j] = 1
                    break
                elif i >=  df['first_min_on'][k].minute and df['last_min_on'][k].hour != Hour:
                    X_aux[i][j] = 1
                    break
                else:
                    X_aux[i][j] = 0
                    break
                    




In [5]:
# DEVICE DRINKS REFRIGERATOR
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Drinks_Refrigerator.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    

    # Use a função read_excel para carregar os dados em um DataFrame.
    df0 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df0 = df0.iloc[:,0:3]
j=0
function_Ton_T_off (df0, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 0:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')



C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

'\n\n\n#T_ON, T_OFF = function_Ton_T_off (df0, j, nome_do_arquivo)\n\n\nfor i in range (T):\n    print(T_ON[i])\n'

In [6]:

# DEVICE BLAST CHILLER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Blast_Chiller.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df1 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df1 = df1.iloc[:,0:3]
j=1
function_Ton_T_off (df1, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 1:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')



C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

In [7]:

# DEVICE GARDE MANGER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Garde_Manger_10.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df2 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df2 = df2.iloc[:,0:3]

j=2
function_Ton_T_off (df2, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 2:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

In [8]:

# DEVICE ICE MACHINE
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Ice_Machine.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df3 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df3 = df3.iloc[:,0:3]
j=3
function_Ton_T_off (df3, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 3:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [9]:

# DEVICE DUAL FRYER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Dual_Fryer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df4 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df4 = df4.iloc[:,0:3]
j=4
function_Ton_T_off (df4, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 4:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [10]:

# DEVICE SALAMANDER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Salamander_1.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df5 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df5 = df5.iloc[:,0:3]
j=5
function_Ton_T_off (df5, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 5:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [11]:

# DEVICE INFRARED SHELF
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Infrared_Shelf.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df6 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df6 = df6.iloc[:,0:3]
j=6
function_Ton_T_off (df6, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 6:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [12]:

# DEVICE GLASS WASHER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Glass_Washer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df7 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df7 = df7.iloc[:,0:3]
j=7
function_Ton_T_off (df7, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 7:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')



C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [13]:

# DEVICE DISH WASHER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Dish_Washer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df8 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df8 = df8.iloc[:,0:3]
j=8
function_Ton_T_off (df8, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 8:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)


i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [14]:

# DEVICE CONVECTION OVEN
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Convection_Oven_1.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df9 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df9 = df9.iloc[:,0:3]
j=9
function_Ton_T_off (df9, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 9:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\4232574009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [15]:
#################################################################################################################
# Objective: Read some appliance's characteristics from the excel "Info".
#
# ->Dev: Set of devices (1 a 13);
# ->Type: 0 (Not Flexibel), 1 (Power Flexibility), 2 (Time Flexibility) and 3 (Time and Power Flexibility);
# ->T_ON: Time that each load needs to be turned ON;
# ->T_OFF: Time that each load needs to be turned ON and OFF;
# ->State_init: Initial state (t=0) of each appliance: 0 (desligada) ou 1 (ligada);
# ->P_max: Maximum Power for each appliance.
# ->W_max: Maximum Weight for each appliance;
# ->W_min: Minimum Weight for each appliance;
# ->Y_max: Maximum Power percentage for each appliance;
# ->Y_min: Minimum Power percentage for each appliance;
# ->T_Hold_Power: Time needed to fix a power change for each appliance.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read 'Set of devices'
column = worksheet['B']
Dev = []
for cell in column:
    if cell.row == 1:
        continue
    Dev.append(cell.value)

# read 'Type of appliances'
column = worksheet['C']
Type = []
for cell in column:
    if cell.row == 1:
        continue
    Type.append(cell.value)



# read 'Initial State'
column = worksheet['D']
State_init = []
for cell in column:
    if cell.row == 1:
        continue
    State_init.append(cell.value)

# read 'Maximum Power'
column = worksheet['E']
P_max = []
for cell in column:
    if cell.row == 1:
        continue
    P_max.append(cell.value)

# read 'Maximum Weight'
column = worksheet['F']
W_max = []
for cell in column:
    if cell.row == 1:
        continue
    W_max.append(cell.value)

# read 'Minimum Weight'
column = worksheet['G']
W_min = []
for cell in column:
    if cell.row == 1:
        continue
    W_min.append(cell.value)

# read 'Maximum percentage of power'
column = worksheet['H']
Y_max = []
for cell in column:
    if cell.row == 1:
        continue
    Y_max.append(cell.value)

# read 'Minimum percentage of power'
column = worksheet['I']
Y_min = []
for cell in column:
    if cell.row == 1:
        continue
    Y_min.append(cell.value)

for j in range(len(Dev)):
    Y_max [j] = Y_max [j] / 100
    Y_min [j] = Y_min [j] / 100

# read 'Time needed to fix a power change'
column = worksheet['J']
T_Hold_Power = []
for cell in column:
    if cell.row == 1:
        continue
    T_Hold_Power.append(cell.value)

# close the workbook
workbook.close()

# Print the values
print(f"Index of devices - {(Dev)}")
print(f"Type of appliances - {(Type)}")
#print(f"T_ON - {(T_ON)}")
#print(f"T_OFF - {(T_OFF)}")
print(f"Initial State - {(State_init)}")
print(f"Maximum Power - {(P_max)}")
print(f"Maximum Weight - {(W_max)}")
print(f"Minimum Weight - {(W_min)}")
print(f"Y_max - {(Y_max)}")
print(f"Y_min - {(Y_min)}")
print(f"T_Hold_Power - {(T_Hold_Power)}")



Index of devices - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Type of appliances - [2, 2, 2, 2, 2, 2, 2, 0, 0, 0]
Initial State - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Maximum Power - [5000, 1500, 1000, 400, 5000, 3000, 800, 1600, 4200, 7000]
Maximum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Minimum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Y_max - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Y_min - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
T_Hold_Power - [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [16]:

# Set of indices in scheduling horizon        
P_lim = 12000 # Power limit      
rows = T # Auxiliary variable
cols = len(Dev) # Auxiliary variable 
P_rlx = [0] * T


Controlo=1

# On/Off variables:
X_onoff_FC_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_opt_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i

# Power variables:
P_sum=[0] * T # Sum of Power of all appliances at time i
Y_onoff_FC_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
Y_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
Y_opt_matrix = [[1 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
P_aux = [0] * T

for i in range(T):
    for j in range(len(Dev)):
        Y_onoff_FC_matrix[i][j] = Y_min[j]


# Weight variables:
W_FC_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
WT_opt_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i

# Auxiliary variables:
cont_on_dataset = [0] * len(Dev)
cont_off_dataset = [0] * len(Dev)
state = [0] * len(Dev) #(0=desligado, 1=ligado)
T_ON_atualizado = [0] * len(Dev)
T_ON_atualizado = T_ON.copy()


Power_aux =[[0 for j in range(cols)] for i in range(rows)]
aux =[[0 for j in range(cols)] for i in range(rows)]
P =[[0 for j in range(cols)] for i in range(rows)]
P_sum_aux = [0] * T

P_opt= [[0 for j in range(cols)] for i in range(rows)]
P_sum_opt = [0] * T
differences = [0] * T

X_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
W_Time_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
W_Time_opt_matrix = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_opt_matrix_real = [['' for j in range(cols)] for i in range(rows)] # Status of appliance j at time i



if Controlo == 0:
    # -------------------------------- CUSTO FIXO ------------------------------------#
    Energy_price = 0.13 # Energy price  

elif Controlo == 1:
    # -------------------------------- CUSTO VARIAVEL ------------------------------------#
    Energy_price = [0] * T
    price_variation = 30
    counter = 0
    for i in range(T):
        if counter >= 0 and counter < price_variation:
            Energy_price[i] = 0.06
            counter = counter + 1
        
        else:
            Energy_price[i] = 0.13
            counter = counter + 1

    print(f"Custo - {(Energy_price)}")




Custo - [0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.06, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13, 0.13]


In [17]:

def function_dataset ():

    """ Before Optimization: """
    for i in range(T):
        for j in range(len(Dev)): 

            # StaFC ON 
            if (i==0 and State_init[j] == 1):     
                X_onoff_FC_matrix[i][j] = 1
                cont_on_dataset[j] += 1
                state[j]= 1
            
            # StaFC OFF
            elif (i==0 and State_init[j] == 0):         
                X_onoff_FC_matrix[i][j] = 0
                cont_off_dataset[j] += 1
                state[j]= 0
        
            # Loads with time flexibility
            # Loads with power flexibility 
            # Loads non-flexible
            if (i!=0):

                if state[j] == 1 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_on_dataset[j] < T_ON[i][j]:                    
                        X_onoff_FC_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j]=1
                    else:                                       
                        cont_on_dataset[j] = 0
                        X_onoff_FC_matrix[i][j] = 0
                        cont_off_dataset[j] = 1
                        state[j]=0

                elif state[j] == 0 and (Type[j]== 2 or Type[j]== 3):                              
                    if cont_off_dataset[j] < T_OFF[i][j]:                    
                        X_onoff_FC_matrix[i][j] = 0
                        cont_off_dataset[j] += 1
                        state[j]=0
                    else:                                       
                        cont_on_dataset[j] = 1
                        X_onoff_FC_matrix[i][j] = 1
                        cont_off_dataset[j] = 0
                        state[j]=1

                elif state[j] == 0 and (Type[j]== 1 or Type[j]== 0):                          
                    if ((Type[j]== 1) and (i==1 or i==13 or i==19*60 or i==20*60 or i==21*60)) or ((Type[j]== 0) and (i==12 or i==23 or i==20*60)): 
                    #Ligar cargas com flex. em potencia (luzes) às 07:00 , 19:00 , 20:00 e 21:00; 
                    #Ligar cargas não flexiveis as 13:00 e 20:00;
                        X_onoff_FC_matrix[i][j] = 1
                        cont_on_dataset[j] += 1
                        state[j] = 1
                    else:
                        X_onoff_FC_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0

                elif state[j] == 1 and (Type[j]== 1 or Type[j]== 0): 
                    if cont_on_dataset[j] < T_ON[i][j]:                    
                        cont_on_dataset[j] += 1
                        X_onoff_FC_matrix[i][j] = 1
                        state[j] = 1
                    else:
                        X_onoff_FC_matrix[i][j] = 0
                        state[j] = 0
                        cont_on_dataset[j] = 0
                       
    return X_onoff_FC_matrix

               

In [18]:

def function_weight (X_onoff_FC_matrix):
   
    # Before Optimization: 
    for i in range (T):

        for j in range(len(Dev)): 

            # Loads with time flexibility
            if ((Type[j]==2 or Type[j]==3)):
                if (i==0 and State_init[j] == 1): # StaFC ON   
                    W_FC_matrix[i][j] = 10 - (9/T_ON[i][j])   # Weight
                    
                elif (i==0 and State_init[j] == 0): # StaFC OFF
                    W_FC_matrix[i][j] = 1 + (9/T_OFF[i][j])              # Weight

            # Loads non-flexible
            # Loads with power flexibility
            if (i==0 and (Type[j] == 0 or Type[j]) ) == 1:
                W_FC_matrix[i][j] = 10                                # Weight


            # Loads with time flexibility
            if i!= 0 and (Type[j]==2 or Type[j]==3):
                if X_onoff_FC_matrix[i][j] == 1 : # State ON                                                                                               
                    W_FC_matrix [i][j] =  W_FC_matrix [i-1][j] - (9/T_ON[i][j] ) # Weight decreases
                if X_onoff_FC_matrix[i][j] == 0 : # State OFF 
                     W_FC_matrix [i][j] =  W_FC_matrix [i-1][j] + (9/T_OFF[i][j] ) # Weight increases
        
            W_FC_matrix [i][j]=( W_FC_matrix [i][j])


    return (W_FC_matrix)

             

In [19]:

def function_Power (Y, X):
    
    # Initialize variables to zero
    for i in range(T):
        for j in range(len(Dev)):  
            Power_aux [i][j] = 0
            

    # Power of each device 
    for i in range(T):
        for j in range(len(Dev)):
            Power_aux [i][j] = P_max [j] * Y[j] * X[i][j]



    return(Power_aux)


In [20]:

def function_Power_Sum (Power_aux):
    
    # Initialize variables to zero
    for i in range(T):
        P_sum_aux [i]= 0

    # Sum of device's power at each time i
    for i in range(T):
        for j in range(len(Dev)):  
            P_sum_aux[i] = P_sum_aux [i] + Power_aux[i][j]

    return(P_sum_aux)


In [21]:

for i in range(T):
    for j in range(len(Dev)):
        X_onoff_FC_matrix[i][j] = X_aux[i][j]
 
#X_onoff_FC_matrix = function_dataset()
W_FC_matrix = function_weight(X_onoff_FC_matrix)  
P = function_Power(Y_min, X_onoff_FC_matrix) 
P_sum = function_Power_Sum(P) 

consumo=0
for i in range(T):
    consumo = consumo + P_sum[i]* Energy_price[i]
#consumo = consumo * Energy_price

In [22]:

model = pe.ConcreteModel()

# Set Declaration
model.loads = pe.Set(initialize=np.arange(len(Dev)))
model.Time = pe.Set(initialize=np.arange(T))

X_opt_initial_values = [[0 for j in model.loads] for i in model.Time]
Y_opt_initial_values = [[1 for j in model.loads] for i in model.Time]
W_time_opt_initial_values = [[1 for j in model.loads] for i in model.Time]
VR_initial_values = [[0 for j in model.loads] for i in model.Time]

# Parameter Declaration
model.P_lim = pe.Param(initialize=P_lim)
model.Type = pe.Param(model.loads, initialize=Type)
model.P_max = pe.Param(model.loads, initialize=P_max)
model.State_init = pe.Param(model.loads, initialize=State_init)
model.T_OFF = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: T_OFF[i][j])
model.T_ON = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: T_ON[i][j])
model.W_max = pe.Param(model.loads, initialize=W_max)
model.W_min = pe.Param(model.loads, initialize=W_min)
model.Y_max = pe.Param(model.loads, initialize=Y_max)
model.Y_min = pe.Param(model.loads, initialize=Y_min)
model.X = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: X_onoff_FC_matrix[i][j])
#model.W_time = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: W_FC_matrix[i][j])
model.Y = pe.Param(model.Time, model.loads, initialize=lambda m, i, j: Y_onoff_FC_matrix[i][j])


# Variable Declaration
model.X_opt = pe.Var(model.Time, model.loads, domain=pe.Binary, initialize=lambda m, i, j: X_opt_initial_values[i][j])
model.W_time_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
model.P_rlx = pe.Var(model.Time, domain=pe.NonNegativeReals, initialize=P_rlx)
#model.W_power_opt_aux = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
#model.W_power_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j:W_time_opt_initial_values [i][j])
#model.Y_opt = pe.Var(model.Time, model.loads, domain=pe.NonNegativeReals, initialize=lambda m, i, j: Y_opt_initial_values[i][j])
model.power = pe.Var(model.Time, domain=pe.NonNegativeReals, initialize=differences)

"""
# -------------------------------- CUSTO FIXO ------------------------------------#

#model.price = pe.Param(initialize=Energy_price)
model.price = pe.Param( initialize=Energy_price)


"""
# -------------------------------- CUSTO VARIAVEL ------------------------------------#
#model.aux = pe.Var(model.Time, domain=pe.Reals, initialize=aux)
model.price = pe.Param(model.Time, domain=pe.NonNegativeReals, initialize=Energy_price)




# Constrait 1
def constraint_one(m, i):
    return sum((m.P_max[j] * m.X_opt[i, j]*m.Y_min[j] ) for j in m.loads) <= m.P_lim + m.P_rlx[i]
model.constraint_one = pe.Constraint(model.Time, rule=constraint_one)


# Constrait 2
def _constraint_two(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.X_opt[i, j] == m.X[i, j]  
    else:
        return pe.Constraint.Feasible
model.constraint_two = pe.Constraint( model.Time, model.loads, rule=_constraint_two)


# Constrait 3
def _constraint_three(m, i, j):
    if m.Type[j] == 2 or m.Type[j] == 3:
        return m.W_time_opt[i,j] <= 10 
    else:   
        return pe.Constraint.Feasible
model.constraint_three = pe.Constraint(model.Time, model.loads, rule=_constraint_three)


# Constrait 4
def _constraint_four(m, i, j):
    if m.Type[j] == 2 or m.Type[j] == 3:
        return 1  <= m.W_time_opt[i,j] 
    else:   
        return pe.Constraint.Feasible
model.constraint_four = pe.Constraint(model.Time, model.loads, rule=_constraint_four)


# Constrait 5
def constraint_five(m, i, j):
    if m.Type[j] == 0 or m.Type[j] == 1:
        return m.W_time_opt[i, j] == 10
    elif m.Type[j] == 2 or m.Type[j] == 3:
        if i==0:
            return m.W_time_opt[i, j] == ((1 + 9/m.T_OFF[i,j]) * (1 - m.X_opt[i, j]) + (10 - 9/m.T_ON[i,j]) * m.X_opt[i, j] )
        elif i!= 0:
            return m.W_time_opt[i, j] ==( (m.W_time_opt[i-1, j] - 9/m.T_ON[i,j]) * m.X_opt[i, j] + (m.W_time_opt[i-1, j] + 9/m.T_OFF[i,j]) * (1 - m.X_opt[i, j]) ) 
        else:   
            return pe.Constraint.Skip
    else:   
        return pe.Constraint.Skip
model.constraint_five = pe.Constraint(model.Time, model.loads, rule=constraint_five)





def _constraint_OF(m, i):
    return m.power[i] == sum( (m.P_max[j]) * m.X_opt[i, j]*m.Y_min[j] for j in range(len(Dev)))
model.constraint_OF = pe.Constraint(model.Time, rule=_constraint_OF)


def _objective_function(m):
    return sum( (m.power[i]* m.price[i]) + (10000*m.P_rlx[i])  for i in m.Time)
model.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)






"""

# -------------------------------- CUSTO FIXO ------------------------------------#

# Calcular a soma fora da restrição
def _summation_rule(m, i):
    return sum(m.P_max[j] * m.price * m.Y_min[j] * m.X_opt[i, j]   for j in range(len(Dev)))
model.summation = pe.Expression(model.Time, rule=_summation_rule)


# Constrait 6
def _constraint_six(m, i):
    return m.fixed_cost[i] == m.summation[i]
model.constraint_six = pe.Constraint(model.Time, rule=_constraint_six)


# Objective function, min (custo fixo)
def _objective_function(m):
    return sum(( m.fixed_cost[i] )  for i in m.Time) 
model.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)





# -------------------------------- CUSTO VARIAVEL ------------------------------------#

# Calcular a soma fora da restrição
def _summation_rule(m, i):
    return sum(m.P_max[j] * m.Y_opt[i, j] * m.X_opt[i, j] for j in range(len(Dev)))

model.summation = pe.Expression(model.Time, rule=_summation_rule)


# Definir a restrição separadamente para cada índice de Time
def _constraint_50_rule(m, i):
    return m.aux[i] == m.price[i] * m.summation[i]

model.constraint_50 = pe.Constraint(model.Time, rule=_constraint_50_rule)



# Objective function, min (custo varivel)
def _objective_function(m):
    return sum(  ( ( m.aux[i] ) + (10000000000*(m.VR_UL[i, j] + m.VR_IL[i, j])) ) for i in m.Time for j in m.loads) + (1000000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)

"""




'\n\n# -------------------------------- CUSTO FIXO ------------------------------------#\n\n# Calcular a soma fora da restrição\ndef _summation_rule(m, i):\n    return sum(m.P_max[j] * m.price * m.Y_min[j] * m.X_opt[i, j]   for j in range(len(Dev)))\nmodel.summation = pe.Expression(model.Time, rule=_summation_rule)\n\n\n# Constrait 6\ndef _constraint_six(m, i):\n    return m.fixed_cost[i] == m.summation[i]\nmodel.constraint_six = pe.Constraint(model.Time, rule=_constraint_six)\n\n\n# Objective function, min (custo fixo)\ndef _objective_function(m):\n    return sum(( m.fixed_cost[i] )  for i in m.Time) \nmodel.objective = pe.Objective(expr=_objective_function, sense=pe.minimize)\n\n\n\n\n\n# -------------------------------- CUSTO VARIAVEL ------------------------------------#\n\n# Calcular a soma fora da restrição\ndef _summation_rule(m, i):\n    return sum(m.P_max[j] * m.Y_opt[i, j] * m.X_opt[i, j] for j in range(len(Dev)))\n\nmodel.summation = pe.Expression(model.Time, rule=_summation

In [23]:
     
print("\n--------------------------------Before Optimization----------------------------------------\n")

print("\nForecast X: ")       
for i in range(T):
    print(X_onoff_FC_matrix [i])

print("\nW_Time: ")
for i in range(T):
    print(W_FC_matrix [i])

print("\nPower: ")
for i in range(T):
    print(f"i=[{i+1}] = {P[i]}")

print("\nY: ")
for i in range(T):
    print(f"i=[{i+1}] = {Y_onoff_FC_matrix[i]}")

print("\nP_sum: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum[i]}")
       
# Solver                                                 
results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
model.pprint()


print("\n-----------------------------Model Results--------------------------------------------\n")

for i in model.Time:
    for j in range(len(Dev)):
        X_matrix [i][j] = model.X[i,j]
        #W_Time_matrix [i][j] = round(model.W_time[i,j],2)
        X_opt_matrix [i][j] = round(model.X_opt[i,j].value) # Status on/off of each loads after optimization
        W_Time_opt_matrix [i][j] = round(model.W_time_opt[i,j].value,2)


print("\nX: ")
for i in range(T):
    print(X_matrix[i])

print("\nW_Time: ")
for i in range(T):
    print(W_FC_matrix[i])  

print("\nX_opt: ")
for i in range(T):
    print(X_opt_matrix [i])

print("\nW_Time_opt: ")
for i in range(T):
    print(W_Time_opt_matrix [i])



OF = model.objective()
print('\nMin differences  =', OF)



# Initialize variables to zero
for i in range(T):
    for j in range(len(Dev)):  
        P_opt [i][j] = 0
        

# Power of each device 
for i in range(T):
    for j in range(len(Dev)):
        P_opt [i][j] = P_max [j] * Y_min[j] * model.X_opt[i,j].value


for i in range(T):
    P_sum_opt [i]= 0

# Sum of device's power at each time i
for i in range(T):
    for j in range(len(Dev)):  
        P_sum_opt[i] = P_sum_opt [i] + P_opt[i][j]

#P_sum_opt = function_Power_Sum(P_opt) 

for i in range(T):
    for j in range(len(Dev)):
        P [i][j] = round(P [i][j],2)
        P_opt [i][j] = round(P_opt [i][j],2)
    P_sum [i] = round(P_sum [i],2)
    P_sum_opt [i] = round(P_sum_opt [i],2)


print("\nPower: ")
for i in range(T):
    print(f"i=[{i+1}] = {P[i]}")

print("\nP_sum: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum[i]}")

print("\nPower Opt: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_opt[i]}")

print("\nP_sum Opt: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_sum_opt[i]}")

consumo_opt=0
for i in range(T):
    consumo_opt = consumo_opt + P_sum_opt[i]* Energy_price[i]
#consumo_opt = consumo_opt * Energy_price



for i in range(T):
    P_rlx [i]= round(model.P_rlx[i].value,2)
print("\nP_rlx: ")
for i in range(T):
    print(f"i=[{i+1}] = {P_rlx[i]}")
    
"""
if Controlo == 1:

    # -------------------------------- CUSTO VARIAVEL ------------------------------------#

    print("\nSomatorio ------:") 
    for i in range(T):
        # Imprimir o valor de model.summation[i]
        print(f"Valor de model.summation[{i}] = {(model.summation[i])}")

    print("\nCusto ------------:")
    for i in range(T):
        print(f"Valor de m.aux[{i}] = {(model.aux[i].value)}")
"""



--------------------------------Before Optimization----------------------------------------


Forecast X: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0,

'\nif Controlo == 1:\n\n    # -------------------------------- CUSTO VARIAVEL ------------------------------------#\n\n    print("\nSomatorio ------:") \n    for i in range(T):\n        # Imprimir o valor de model.summation[i]\n        print(f"Valor de model.summation[{i}] = {(model.summation[i])}")\n\n    print("\nCusto ------------:")\n    for i in range(T):\n        print(f"Valor de m.aux[{i}] = {(model.aux[i].value)}")\n'

In [24]:

import datetime

col = []
for i in range(T):
    # Crie um objeto datetime representando o tempo
    time_value = datetime.datetime(1, 1, 1, i // 60, i % 60)

    # Formate o objeto datetime como uma string no formato desejado (hora:minuto:segundo)
    time_str = time_value.strftime("%H:%M:%S")

    col.append(time_str)


for i in range(T):
    X_matrix [i].insert(0, col[i])
    W_FC_matrix [i].insert(0, col[i])

    X_opt_matrix [i].insert(0, col[i])
    W_Time_opt_matrix[i].insert(0, col[i])

    P [i].insert(0, col[i])
    P_opt [i].insert(0, col[i])

    Y_matrix [i].insert(0, col[i])
    Y_opt_matrix [i].insert(0, col[i])

    T_ON [i].insert(0, col[i])
    T_OFF [i].insert(0, col[i])



for i in range(T):
    X_matrix[i].append(0)
    X_matrix[i].append(0)
    X_matrix[i].append(0)

    X_opt_matrix[i].append(0)
    X_opt_matrix[i].append(0)
    X_opt_matrix[i].append(0)

    
    P[i].append(0)
    P[i].append(0)
    P[i].append(0)
    
    P_opt[i].append(0)
    P_opt[i].append(0)
    P_opt[i].append(0)

    Y_matrix[i].append(0)
    Y_matrix[i].append(0)
    Y_matrix[i].append(0)

    Y_opt_matrix[i].append(0)
    Y_opt_matrix[i].append(0)
    Y_opt_matrix[i].append(0)
    

    #change value in ON_or_OFF list in position 16 for sum[i]
    X_matrix[i][11] = P_sum[i]
    X_matrix[i][12] = P_sum_opt[i]
    X_matrix[i][13] = P_rlx [i]

    X_opt_matrix[i][11] = P_sum[i]
    X_opt_matrix[i][12] = P_sum_opt[i]
    X_opt_matrix[i][13] = P_rlx [i]
    
    P[i][11] = P_sum[i]
    P[i][12] = P_sum_opt[i]
    P[i][13] = P_rlx [i]
    
    P_opt[i][11] = P_sum[i]
    P_opt[i][12] = P_sum_opt[i]
    P_opt[i][13] = P_rlx [i]

    Y_matrix[i][11] = P_sum[i]
    Y_matrix[i][12] = P_sum_opt[i]
    Y_matrix[i][13] = P_rlx [i]

    Y_opt_matrix[i][11] = P_sum[i]
    Y_opt_matrix[i][12] = P_sum_opt[i]
    Y_opt_matrix[i][13] = P_rlx [i]
    

row = []
for i in range(len(Dev)+4):
    row.append(i)
row [0] = 'H:M:S / Index'
row [11] ='Sum Power'
row [12] ='Power Optim'
row [13] ='Power Rlx'

row1 = []
for i in range(len(Dev)+1):
    row1.append(i)
row1 [0] = 'H:M:S / Index'

X_matrix.insert(0, row)
W_FC_matrix.insert(0, row1)
X_opt_matrix.insert(0, row)
W_Time_opt_matrix.insert(0, row1)
P.insert(0, row)
P_opt.insert(0, row)
Y_matrix.insert(0, row)
Y_opt_matrix.insert(0, row)

T_ON.insert(0, row1)
T_OFF.insert(0, row1)


In [25]:

# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(X_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='X before Opt', index=False, header=False)

df2 = pd.DataFrame(W_FC_matrix)
df2.to_excel(writer, sheet_name='W_Time before Opt', index=False, header=False)

df3 = pd.DataFrame(X_opt_matrix)
df3.to_excel(writer, sheet_name='X Opt', index=False, header=False)

df4 = pd.DataFrame(W_Time_opt_matrix)
df4.to_excel(writer, sheet_name='W_Time opt', index=False, header=False)

df5 = pd.DataFrame(P)
df5.to_excel(writer, sheet_name='P before Opt', index=False, header=False)

df6 = pd.DataFrame(P_opt)
df6.to_excel(writer, sheet_name='P Opt', index=False, header=False)

df7 = pd.DataFrame(Y_matrix)
df7.to_excel(writer, sheet_name='Y before Opt', index=False, header=False)

df8 = pd.DataFrame(Y_opt_matrix)
df8.to_excel(writer, sheet_name='Y Opt', index=False, header=False)

df9 = pd.DataFrame(T_ON)
df9.to_excel(writer, sheet_name='T ON', index=False, header=False)

df10 = pd.DataFrame(T_OFF)
df10.to_excel(writer, sheet_name='T OFF', index=False, header=False)



writer.save()
writer.close()

C:\Users\anaol\AppData\Local\Temp\ipykernel_7752\1806161763.py:47: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\anaol\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
